#Importing the neccessary libraries

In [20]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

Importing the dataset

In [21]:
df_customers = pd.read_csv('Customers.csv')
df_products = pd.read_csv('Products.csv')
df_transactions = pd.read_csv('Transactions.csv')

Preprocessing the datasets

In [22]:
ls_name = ["Customers", "Products", "Transactions"]
ls = [df_customers, df_products, df_transactions]
for l in ls:
  print(l.info(),"\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ProductID    100 non-null    object 
 1   ProductName  100 non-null    object 
 2   Category     100 non-null    object 
 3   Price        100 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           ----------

Converting the date columns to datetime objects

In [23]:
df_customers['SignupDate'] = pd.to_datetime(df_customers['SignupDate'])
df_transactions['TransactionDate'] = pd.to_datetime(df_transactions['TransactionDate'])

Looking for redundant words

In [24]:
print("Customers", df_customers.duplicated().sum())
print("Products", df_products.duplicated().sum())
print("Transactions", df_transactions.duplicated().sum())

Customers 0
Products 0
Transactions 0


In [25]:
for l in ls:
  print(l.head(),"\n")

  CustomerID        CustomerName         Region SignupDate
0      C0001    Lawrence Carroll  South America 2022-07-10
1      C0002      Elizabeth Lutz           Asia 2022-02-13
2      C0003      Michael Rivera  South America 2024-03-07
3      C0004  Kathleen Rodriguez  South America 2022-10-09
4      C0005         Laura Weber           Asia 2022-08-15 

  ProductID              ProductName     Category   Price
0      P001     ActiveWear Biography        Books  169.30
1      P002    ActiveWear Smartwatch  Electronics  346.30
2      P003  ComfortLiving Biography        Books   44.12
3      P004            BookWorld Rug   Home Decor   95.69
4      P005          TechPro T-Shirt     Clothing  429.31 

  TransactionID CustomerID ProductID     TransactionDate  Quantity  \
0        T00001      C0199      P067 2024-08-25 12:38:23         1   
1        T00112      C0146      P067 2024-05-27 22:23:54         1   
2        T00166      C0127      P067 2024-04-25 07:38:55         1   
3        T0027

Merging the transactions with product details to get more information of category

In [26]:
df_transactions = df_transactions.merge(df_products, on='ProductID', how='left')

In [27]:
df_transactions.head(10)

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
5,T00442,C0188,P067,2024-12-26 14:40:03,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
6,T00490,C0195,P067,2024-11-24 11:49:48,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
7,T00536,C0008,P067,2024-09-22 06:13:59,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
8,T00564,C0157,P067,2024-12-07 17:57:40,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
9,T00631,C0130,P067,2024-05-14 23:14:59,2,601.36,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68


Finding which categories of product customers in the dataset are pu

In [28]:
customerPurchases = df_transactions.groupby(['CustomerID','Category'])['Quantity'].sum().unstack(fill_value=0)

In [29]:
customerPurchases.head(10)


Category,Books,Clothing,Electronics,Home Decor
CustomerID,,,,
C0001,2,0,7,3
C0002,0,4,0,6
C0003,0,4,4,6
C0004,8,0,6,9
C0005,0,0,4,3
C0006,5,4,0,3
C0007,0,0,4,4
C0008,1,8,5,6
C0009,0,2,1,0


Normalizing it ti get an idea of customer's preferences rather than total spending

In [30]:
from sklearn.preprocessing import normalize

customerPurchases_normalized = normalize(customerPurchases)
customerPurchases_normalized = pd.DataFrame(customerPurchases_normalized, columns = customerPurchases.columns, index = customerPurchases.index)
customerPurchases_normalized.head(20)


Category,Books,Clothing,Electronics,Home Decor
CustomerID,,,,
C0001,0.254000,0.000000,0.889001,0.381000
C0002,0.000000,0.554700,0.000000,0.832050
C0003,0.000000,0.485071,0.485071,0.727607
C0004,0.594635,0.000000,0.445976,0.668965
C0005,0.000000,0.000000,0.800000,0.600000
C0006,0.707107,0.565685,0.000000,0.424264
C0007,0.000000,0.000000,0.707107,0.707107
C0008,0.089087,0.712697,0.445435,0.534522
C0009,0.000000,0.894427,0.447214,0.000000


Computing the similarity matrix

In [31]:
smatrix = cosine_similarity(customerPurchases_normalized.fillna(0))

Finding similarity between purchase habits among customers

In [32]:
s_df = pd.DataFrame(smatrix, index=customerPurchases_normalized.index, columns=customerPurchases_normalized.index)

In [33]:
s_df.head(10)

CustomerID,C0001,C0002,C0003,C0004,C0005,C0006,C0007,C0008,C0009,C0010,...,C0191,C0192,C0193,C0194,C0195,C0196,C0197,C0198,C0199,C0200
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0001,1.000000,0.317011,0.708447,0.802387,0.939801,0.341250,0.898027,0.622274,0.397573,0.049814,...,0.440803,0.846830,0.254000,0.826184,0.568455,0.377171,0.965535,0.397573,0.738350,0.378670
C0002,0.317011,1.000000,0.874475,0.556612,0.499230,0.666795,0.588348,0.840083,0.496139,0.543928,...,0.000000,0.108786,0.000000,0.706018,0.956932,0.902134,0.372104,0.496139,0.744208,0.796333
C0003,0.708447,0.874475,1.000000,0.703074,0.824621,0.583095,0.857493,0.950699,0.650791,0.475651,...,0.105227,0.475651,0.000000,0.891793,0.972512,0.788893,0.759257,0.650791,0.867722,0.749940
C0004,0.802387,0.556612,0.703074,1.000000,0.758160,0.704288,0.788382,0.609205,0.199447,0.116618,...,0.677221,0.699705,0.594635,0.882988,0.686194,0.725312,0.698064,0.199447,0.797787,0.607415
C0005,0.939801,0.499230,0.824621,0.758160,1.000000,0.254558,0.989949,0.677062,0.357771,0.000000,...,0.173544,0.627572,0.000000,0.791960,0.727354,0.424264,0.983870,0.357771,0.894427,0.353381
C0006,0.341250,0.666795,0.583095,0.704288,0.254558,1.000000,0.300000,0.692935,0.505964,0.693375,...,0.690268,0.526965,0.707107,0.780000,0.619818,0.920000,0.189737,0.505964,0.379473,0.937043
C0007,0.898027,0.588348,0.857493,0.788382,0.989949,0.300000,1.000000,0.692935,0.316228,0.000000,...,0.153393,0.554700,0.000000,0.800000,0.791257,0.500000,0.948683,0.316228,0.948683,0.390434
C0008,0.622274,0.840083,0.950699,0.609205,0.677062,0.692935,0.692935,1.000000,0.836660,0.716328,...,0.183594,0.541614,0.089087,0.894516,0.905508,0.818923,0.637455,0.836660,0.677296,0.875584
C0009,0.397573,0.496139,0.650791,0.199447,0.357771,0.505964,0.316228,0.836660,1.000000,0.877058,...,0.097014,0.526235,0.000000,0.632456,0.542137,0.505964,0.400000,1.000000,0.200000,0.740797


Function to find top 3 most similar customers for top 20 customers (C0001 to C0020)

In [34]:
def get_top_lookalikes(customer_id, similarity_df, top_n=3):
    if customer_id not in similarity_df.index:
        return "Customer not found"
    similar_customers = (similarity_df.loc[customer_id].drop(customer_id).sort_values(ascending=False).head(top_n))
    lookalike_df = pd.DataFrame({'Customer_ID': similar_customers.index,'Similarity_Score': similar_customers.values})
    csv_filename = "Aviral_Soni_Lookalike.csv"
    lookalike_df.to_csv(csv_filename, index=False)

In [46]:
def lookalikes(s_df, customer_range=('C0001', 'C0020'), top_n=3):
    lookalike_map = {}
    for customer_id in s_df.index:
        if customer_range[0] <= customer_id <= customer_range[1]:
            similar_customers = (
                s_df.loc[customer_id].drop(customer_id).sort_values(ascending=False).head(top_n)
            )
            lookalike_map[customer_id] = [
                (similar_id, score) for similar_id, score in similar_customers.items()
            ]
    return lookalike_map

In [48]:
def save_lookalikes(lookalike_map, filename="Aviral_Soni_Lookalike.csv"):
    lookalike_list = []

    for customer_id, lookalikes in lookalike_map.items():
        for similar_id, score in lookalikes:
            lookalike_list.append({
                'Customer_ID': customer_id,
                'Lookalike_Customer_ID': similar_id,
                'Similarity_Score': score
            })

    lookalike_df = pd.DataFrame(lookalike_list)
    lookalike_df.to_csv(filename, index=False)
    print(f"Lookalike data saved to {filename}")

In [53]:
forcsv=lookalikes(s_df, customer_range=('C0001', 'C0020'), top_n=3)

In [54]:
save_lookalikes(forcsv, filename="Aviral_Soni_Lookalike.csv")

Lookalike data saved to Aviral_Soni_Lookalike.csv
